# Análise de Faturamento e Oportunidades para a Rede de Açaí

## 1. Objetivo da Análise

**Proposta do Stakeholder:** Identificar oportunidades para aumentar o faturamento da franquia.
O principal objetivo deste projeto é analisar o histórico de vendas da rede para identificar padrões, tendências e anomalias, com o intuito de fornecer recomendações estratégicas que ajudem a **aumentar o faturamento e o crescimento do negócio.**

## 2. Perguntas-Chave

Para guiar nossa investigação, definimos as seguintes perguntas:

* **Desempenho:** Quais lojas geram maior faturamento? Quais cidades geram maior faturamento?
* **Tendências:** Qual a sazonalidade de vendas ao ano? O volume está crescendo ao longo do tempo? O crescimento é o mesmo para todas as lojas?
* **Comportamento do Consumidor:** Qual a proporção de vendas entre consumo no local e delivery? Esse padrão muda entre as lojas? Dentre as opções de produtos, qual é a mais vendida?

## 3. Análise Exploratória com Pandas

Nesta etapa, vamos carregar os dados em um DataFrame do Pandas para realizar a análise exploratória.

In [8]:
# Importar a biblioteca Pandas
import pandas as pd

# Carregar o dataset
df = pd.read_excel('src/vendas.xlsx')

# Garantir que a coluna 'data' está no formato de data
df['data'] = pd.to_datetime(df['data'])

# --- Insight 1: Faturamento Total ---
faturamento_total = df['preco'].sum()
print(f"Faturamento Total da Rede: R${faturamento_total:,.2f}\n")

# --- Insight 2: Ranking de Lojas por Faturamento ---
faturamento_por_loja = df.groupby('cidade')['preco'].sum().sort_values(ascending=False)
print("--- Faturamento por Loja ---")
print(faturamento_por_loja)
print("\n")

# --- Insight 3: Análise de Sazonalidade (Melhor e Pior Mês) ---
df['mes'] = df['data'].dt.month_name()
faturamento_por_mes = df.groupby('mes')['preco'].sum().sort_values(ascending=False)
print("--- Faturamento por Mês (Sazonalidade) ---")
print(faturamento_por_mes)
print("\n")

# --- Insight 4: Análise de Crescimento (2022) ---
# Filtrar dados para 2022 e agrupar por loja para ver a tendência
df_2022 = df[df['data'].dt.year == 2022]
crescimento_2022_por_loja = df_2022.groupby('cidade')['preco'].sum().sort_values(ascending=False)
print("--- Faturamento por Loja em 2022 ---")
print(crescimento_2022_por_loja)
print("\n")

# --- Insight 5: Estabilidade de São Paulo em 2022 ---
media_mensal_sp_2022 = df_2022[df_2022['cidade'] == 'São Paulo'].groupby(df_2022['data'].dt.month)['preco'].sum().mean()
print(f"Média mensal de faturamento de São Paulo em 2022: R${media_mensal_sp_2022:,.2f}\n")


# --- Insight 6: Análise do Canal de Venda (Local vs. Delivery) ---
faturamento_por_canal = df.groupby('local_consumo')['preco'].sum()
porcentagem_por_canal = (faturamento_por_canal / faturamento_total) * 100
print("--- Faturamento por Canal de Venda ---")
print(porcentagem_por_canal)

Faturamento Total da Rede: R$584,864.00

--- Faturamento por Loja ---
cidade
Santos            112379
Florianópolis     109185
Fortaleza         103162
São Paulo         102189
Rio de Janeiro     88357
Niterói            69592
Name: preco, dtype: int64


--- Faturamento por Mês (Sazonalidade) ---
mes
March        52977
July         51369
October      50865
December     50098
May          50001
January      48316
August       48078
November     47852
June         47723
September    47165
April        45995
February     44425
Name: preco, dtype: int64


--- Faturamento por Loja em 2022 ---
cidade
Florianópolis     70927
Fortaleza         51822
São Paulo         51332
Santos            46240
Rio de Janeiro    38286
Niterói           34971
Name: preco, dtype: int64


Média mensal de faturamento de São Paulo em 2022: R$4,277.67

--- Faturamento por Canal de Venda ---
local_consumo
Consumo no local    50.228942
Delivery            49.771058
Name: preco, dtype: float64


## 4. Principais Observações da Análise Exploratória

Após uma análise inicial dos dados, compilamos as seguintes observações-chave:

| Métrica/Dimensão      | Observação                                                                                                                              |
| --------------------- | --------------------------------------------------------------------------------------------------------------------------------------- |
| **Faturamento Geral** | O faturamento total da rede no período foi de aproximadamente **R$584k**.                                                               |
| **Performance/Loja** | A loja de **Santos** é a que mais fatura (~R$112k), enquanto **Niterói** é a que menos fatura (~R$70k).                                  |
| **Estabilidade** | A loja de **São Paulo** demonstrou a maior estabilidade de faturamento em 2022, com uma média consistente de R$4,2k por mês.             |
| **Crescimento** | As vendas gerais da rede cresceram significativamente a partir de 2022, mas as lojas de **Santos e Rio de Janeiro** não acompanharam.     |
| **Canal de Venda** | As vendas são divididas quase perfeitamente entre **Consumo no Local (51%) e Delivery (49%)** em todas as lojas.                          |

*Estes insights foram a base para aprofundarmos a análise com visualizações no Power BI.*

## 5. Visualização de Dados e Recomendações Estratégicas

Para aprofundar nossa análise, criamos uma série de visuais no Power BI. Cada gráfico nos ajudou a responder uma pergunta específica e a construir nossas recomendações finais.

### Análise 1: Qual a real performance de cada loja?

Para entender a performance de cada loja ao longo do tempo, criamos um gráfico de linhas do faturamento por cidade.

![Faturamento por Cidade](src/faturamento_cidade.png)

Este gráfico revelou padrões muito distintos. Enquanto algumas lojas mostram estabilidade (São Paulo), outras são muito voláteis (Santos) ou estão em declínio (Rio de Janeiro). Isso nos levou a focar na comparação entre a loja mais e menos estável.

![Comparação Niterói vs. Santos](src/niteroi_santos.png)

* **Insight:** A análise confirmou que a estabilidade não está ligada ao faturamento total. A loja de Santos, apesar de faturar mais, tem picos e quedas bruscas, sugerindo um problema de retenção de clientes.
* **Recomendação:** Focar na qualidade do serviço em lojas voláteis para aumentar a lealdade do cliente.

### Análise 2: Quais fatores NÃO explicam a diferença de performance?

Para entender se o sucesso era devido ao *que* era vendido ou *como*, investigamos o mix de produtos e os canais de venda.

![Mix de Produtos por Cidade](src/produtos_cidade.png)
*Gráfico de Produtos Mais Vendidos por Cidade*

![Mix de Canais por Cidade](src/metodo_consumo.png)
*Gráfico de Proporção de Consumo Local vs. Delivery por Cidade*

* **Insight:** Surpreendentemente, tanto o mix de produtos quanto a proporção de vendas por delivery são extremamente consistentes em todas as lojas. Isso significa que a causa da boa ou má performance não está no produto nem no canal, mas sim na operação de cada loja.
* **Recomendação:** Já que o delivery é um pilar de 50% em todas as lojas, investir em marketing neste canal trará benefícios para toda a rede.

### Análise 3: Existe um padrão de Sazonalidade?

Finalmente, investigamos se as vendas eram influenciadas pela época do ano.

![Faturamento por Mês](src/faturamento_mes.png)

* **Insight:** O faturamento mensal é relativamente estável, sem picos ou vales drásticos. Isso reforça a ideia de que a base de clientes é leal e o negócio não depende de fatores sazonais.
* **Recomendação:** A estratégia da empresa não precisa se preocupar tanto com promoções para "meses fracos", podendo focar em estratégias de crescimento consistentes ao longo do ano.

---
### Conclusão Final

Com base na nossa análise, identificamos três grandes oportunidades para aumentar o faturamento e a estabilidade da rede:

**Foco no Serviço ao Cliente:** Nossa maior oportunidade não está no que vendemos, mas em como vendemos. As lojas mais estáveis, como a de São Paulo, provavelmente retêm clientes através de um serviço excelente. Recomendamos conduzir pesquisas de satisfação, começando por Santos e Rio de Janeiro, para entender e corrigir as falhas que causam a instabilidade no faturamento.

**Explorar o Potencial do Delivery:** O delivery já representa 50% do nosso faturamento em todas as lojas, o que mostra que ele é um pilar do negócio. Recomendamos investir ativamente neste canal, com campanhas e anúncios em aplicativos de entrega, para transformar essa base sólida em um motor de crescimento.

**Replicar o Sucesso do Passado:** Identificamos um pico de vendas em toda a rede no final de 2021. É crucial descobrirmos que campanha de marketing foi essa e replicá-la, pois já provou que tem a capacidade de aumentar o faturamento de forma generalizada.

A análise dos dados nos mostra que o sucesso da rede não está em criar novos produtos, mas em otimizar a operação e a estratégia de marketing. As recomendações são: **melhorar a consistência do serviço**, **investir no crescimento do canal de delivery** e **replicar campanhas de marketing que já se provaram eficazes.**